In [1]:
import sys
sys.path.append('/home/monte.flora/python_packages/Faster-LIME')
#from faster_lime.explainers.numpy_tabular_explainer import NumpyTabularExplainer
#from faster_lime.explainers.numpy_robust_tabular_explainer import NumpyRobustTabularExplainer

from lime.lime_tabular import LimeTabularExplainer 

import sys, os 
from os.path import dirname
path = dirname(dirname(os.getcwd()))
sys.path.insert(0, path)
sys.path.insert(0, '/home/monte.flora/python_packages/scikit-explain')

import skexplain 
from skexplain.common.importance_utils import to_skexplain_importance
from skexplain.common.multiprocessing_utils import run_parallel, to_iterator
from src.io.io import load_data_and_model
from src.common.util import subsampler, normalize_importance, compute_sage
import pandas as pd
from skexplain.main.lime_fast import FastLimeTabularExplainer

In [2]:
sys.path.append('/home/monte.flora/python_packages/scikit-explain')
import skexplain
### Loading the training data and pre-fit models 
estimators = skexplain.load_models()
X,y = skexplain.load_data()

In [3]:
features=list(X.columns)

In [8]:
%%time
lime_kws = {'training_data': X.values, 'categorical_names': ['urban', 'rural']}
X_sub, y_sub = subsampler(X,y, 100)
explainer = skexplain.ExplainToolkit(estimators[0], X_sub, y_sub)
explainer.local_attributions('lime', lime_kws=lime_kws, n_jobs=1)

LIME:  59%|███████████████████████████████████████████████████████████▋                                         | 5907/10000 [06:04<04:12, 16.21it/s]


KeyboardInterrupt: 

In [5]:
%%time
X = X.astype({'urban': int, 'rural': int})

explainer = FastLimeTabularExplainer(
        training_data=X.values,
        feature_names=features,
        categorical_names=['rural', 'urban']
    )

exp = explainer.explain_instance(
        data_row=X.values[0,:],
        predict_fn=estimators[0][1].predict_proba, 
        num_features=X.shape[1],
        num_samples=2500, 
        label=1,
    )

print(exp)
#data = pd.DataFrame({f : [contrib] for f, contrib in exp[0]})
# Make sure the contributions are in the correct order. 
#data = data[features]

(array([ 0.01590385, -0.01350816, -0.00556447, -0.00385753,  0.00070345,
        0.00476732, -0.00024513, -0.00186269,  0.00279847,  0.01019066,
       -0.01180746,  0.01529637, -0.0007805 ,  0.0172287 , -0.00490622,
        0.00821469, -0.00525848,  0.00185144,  0.00144046,  0.00427252,
       -0.0014761 , -0.00025344, -0.00407711, -0.01962   , -0.00189342,
        0.00347003, -0.00085746, -0.00509191, -0.00428615, -0.0025716 ]), 0.39271621722055267)
CPU times: user 258 ms, sys: 41.9 ms, total: 300 ms
Wall time: 200 ms


In [6]:
data

NameError: name 'data' is not defined

In [ ]:
%%time

lime_kws = {"training_data" : X.values, "mode" : "classification", "categorical_features" : ['urban', 'rural']}

explainer = LimeTabularExplainer(**lime_kws)
exp = explainer.explain_instance(
        data_row=X.values[0,:],
        predict_fn=estimators[0][1].predict_proba, 
        num_features= X.shape[1],
        num_samples=2500,
    )

print(exp)

In [ ]:
isinstance(explainer, LimeTabularExplainer)

In [ ]:
# 10 features = 6.8 s
# 30 features 